In [27]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from collections import defaultdict

In [28]:
##Path of chromedriver
chromedriver="C:/Users/k.shridhar/Documents/chromedriver.exe"

In [30]:
cricinfo_match_notes_url='https://www.espncricinfo.com/series/19430/scorecard/1183533/australia-vs-new-zealand-2nd-test-icc-world-test-championship-2019-2021'

In [29]:
cricbuzz_match_url='https://www.cricbuzz.com/cricket-scores/22582/aus-vs-nz-2nd-test-new-zealand-tour-of-australia-2019-20'

In [31]:
driver = webdriver.Chrome(executable_path=chromedriver)
driver.get(cricinfo_match_notes_url)
cricinfo_matchnotes_soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()
cricinfo_matchnotes_soup

<html class="js no-touch history cssanimations video localstorage desktop-ua scrollbars-17 desktop" lang="en" style=""><head><script async="async" charset="UTF-8" src="https://cdn.taboola.com/libtrc/userx.20200203-21-RELEASE.es6.js" type="text/javascript"></script><script async="async" charset="UTF-8" src="https://cdn.taboola.com/libtrc/feed-card-placeholder.20200203-21-RELEASE.es6.js" type="text/javascript"></script><script src="https://www.googletagservices.com/activeview/js/current/osd.js?cb=%2Fr20100101"></script><script async="" data-jsonpid="" src="https://cdn-gl.imrworldwide.com/novms/js/2/nlsSDK600.bundle.min.js"></script><script async="" src="https://cdn-gl.imrworldwide.com/conf/PED8CDAC2-F114-41BE-8B98-AFA06FAEA06E.js#name=cricinfo/cricket/1183533&amp;ns=NOLBUNDLE"></script><script charset="UTF-8" src="https://cdn.taboola.com/libtrc/impl.20200203-21-RELEASE.js" type="text/javascript"></script><script async="" src="https://sb.scorecardresearch.com/beacon.js"></script><script s

In [40]:
m_notes=cricinfo_matchnotes_soup.find('h1',text='Match Notes')
all_days=[d for d in m_notes.next_element.next.find_all('ul',{'class':'bulleted-list'})]
all_days.reverse()
all_days

[<ul class="bulleted-list" data-reactid="1343"><li data-reactid="1344">Australia 1st innings</li><li data-reactid="1345">Drinks: Australia - 31/1 in 13.0 overs (DA Warner 21, M Labuschagne 8)</li><li data-reactid="1346">Australia: 50 runs in 19.1 overs (115 balls), Extras 2</li><li data-reactid="1347">2nd Wicket: 50 runs in 111 balls (DA Warner 33, M Labuschagne 15, Ex 2)</li><li data-reactid="1348">Lunch: Australia - 67/2 in 26.0 overs (M Labuschagne 23, SPD Smith 1)</li><li data-reactid="1349">Drinks: Australia - 99/2 in 40.0 overs (M Labuschagne 35, SPD Smith 21)</li><li data-reactid="1350">Australia: 100 runs in 40.1 overs (241 balls), Extras 2</li><li data-reactid="1351">3rd Wicket: 50 runs in 118 balls (M Labuschagne 28, SPD Smith 27, Ex 0)</li><li data-reactid="1352">M Labuschagne: 50 off 132 balls (4 x 4, 1 x 6)</li><li data-reactid="1353">Attendance: 80,473</li><li data-reactid="1354">Australia: 150 runs in 53.1 overs (319 balls), Extras 3</li><li data-reactid="1355">Tea: Aust

In [42]:
def create_innings_df(all_days):
    '''Process list of day wise reviews andreturn a neater dataframe'''
    innings_list=[] 
    innings_reviews=defaultdict(list)
    innings_list.append([a.text for a in all_days[0] if "innings" in a.text and len(a.text)<=30])
    ##There are instances where cricinfo match notes has the word 'innings' which might not be due to an innings beginning
    
    day_wise_reviews=[a.text for a in all_days[0] if a.text.startswith("Over") or "innings" in a.text]
    ##Append from day2 onwards to the day 1 list
    
    if len(all_days)>=2:
        for ad in all_days[1:]:
            daylist=[a.text for a in ad if a.text.startswith("Over") or "innings" in a.text]
            innings_list.append([a.text for a in ad if "innings" in a.text and len(a.text)<=30])
            for d in daylist:
                day_wise_reviews.append(d)
    
    idxs = [i for i,x in enumerate(day_wise_reviews) if 'innings' in x]
    start_end_idxs=list(map(list, zip(idxs, idxs[1:])))
    for s in start_end_idxs:
        innings_reviews[day_wise_reviews[s[0]]]=day_wise_reviews[s[0]+1:s[1]]
    innings_reviews[day_wise_reviews[max(idxs)]]=day_wise_reviews[max(idxs)+1:]
    
    ##Make the innings_wise_reviews dictionary to dataframe
    
    idf=pd.DataFrame.from_dict(innings_reviews,orient='index')
    idf.reset_index(inplace=True)
    idf.fillna('',inplace=True)
    innings_df=idf.melt(id_vars='index',value_name='reviews')
    
    ##In this case, the variable column does not add any value. Hence it can be dropped
    innings_df.drop(columns='variable',inplace=True) 
    innings_df.columns=['innings','reviews']
    innings_df=innings_df[innings_df.reviews!='']
    
    ###Augment Innings_df with the referral notes
    
    over=[]
    review_team=[]
    review_umpire=[]
    review_batsman=[]
    review_outcome=[]
    
    
    for review in innings_df.reviews:
        over.append(review.split('Over')[1].split(':')[0].strip())
        review_team.append(review.split(':')[1].strip().split('by ')[1].split(',')[0])
        review_umpire.append(review.split(':')[1].strip().split('Umpire - ')[1].split(',')[0])
        review_batsman.append(review.split(':')[1].strip().split('Batsman -')[1].strip().split('(')[0].strip())
        review_outcome.append(review.split(':')[1].strip().split('Batsman -')[1].strip().split('(')[1].split(')')[0].strip())

    innings_df['Over']=over
    innings_df['Review_team']=review_team
    innings_df['Review_batsman']=review_batsman
    innings_df['Review_umpire']=review_umpire
    innings_df['Review_outcome']=review_outcome
    innings_df['Umpires_call']=innings_df['Review_outcome'].apply(lambda x:"Umpire" in x)
    innings_df['index']=range(len(innings_df.reviews))
    innings_list_updated=[]
    for i in innings_list:
        if(i):
            for a in range(len(i)):
                innings_list_updated.append(i[a])
            
    ##Remove duplicates if any, from innings list 
    
    innings_list_updated=list(dict.fromkeys(innings_list_updated))
    innings_list_updated.reverse()
    
    innings_df.set_index('index',inplace=True,drop=False)
    return innings_df,innings_list_updated


In [47]:
idf,idu=create_innings_df(all_days)

In [60]:
fow_text=[fow.text for fow in cricinfo_matchnotes_soup.find_all('div',{"class":"wrap dnb"}) if "Fall of wickets:" in fow.text]
innings_fow=defaultdict(list)

for a,inn in enumerate(reversed(idu)):
    innings_fow[inn]=[f.split(')')[0].strip().split(' ')[0] for i,f in enumerate(fow_text[a].split(':')[1].strip().split(',')) if i%2!=0]

innings_fow_df=pd.DataFrame.from_dict(innings_fow,orient='index')
innings_fow_df.reset_index(inplace=True)
innings_fow_df.fillna('',inplace=True)
innings_fow_df=innings_fow_df.melt(id_vars='index',value_name='wickets')
##In this case, the variable refers to the fall of wicket. 

innings_fow_df['variable']=innings_fow_df['variable']+1
innings_fow_df.columns=['innings','active_partnership','Over']
innings_fow_df=innings_fow_df[innings_fow_df.Over!='']
pbreak_innings=pd.merge(innings_df,innings_fow_df,on=['innings','Over'],how='inner')['index']
innings_df['Partnership_broken']=False
innings_df.loc[pbreak_innings,'Partnership_broken']=True
# innings_df,innings_fow_df

In [49]:
driver = webdriver.Chrome(executable_path=chromedriver)
driver.get(cricbuzz_match_url)
cricbuzz_match_soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()
cricbuzz_match_soup


<html itemscope="" itemtype="http://schema.org/WebPage" lang="en"><head><style type="text/css">@charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}.ng-animate-shim{visibility:hidden;}.ng-anchor{position:absolute;}</style><meta charset="utf-8"/><script async="" src="https://sb.scorecardresearch.com/c2/6036484/cs.js" type="text/javascript"></script><script async="" src="https://www.googletagservices.com/tag/js/gpt.js" type="text/javascript"></script><script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script async="" src="//www.googletagmanager.com/gtm.js?id=GTM-PGNCT7"></script><script async="" src="/dist/js/cricbuzz.min.6556.js" type="text/javascript"></script><script>var is_mobile = /symbian|tizen|midp|uc(web|browser)|MSIE (5.0|6.0|7.0|8.0)|tablet/i.test(navigator.userAgent);	if(is_mobile && window.location.hostname != 

### Process cricbuzz commentary to map innings and Over

In [69]:

commentary_text=[c.text for c in cricbuzz_match_soup.find_all('p',{'class':'cb-col cb-col-90 cb-com-ln'})]
over_text=[o.text for o in cricbuzz_match_soup.find_all('span',{'cb-col cb-col-8 text-bold'})]
inngs_breaks=[index for index, value in enumerate(over_text) if value == '0.1']
current_innings=[]
current_commentary=[]
current_over=[]
end=0
for i,b in enumerate(ib):
    start=b
    for on,o in enumerate(over_text[end:start+1]):
        current_innings.append(idu[i])
        current_over.append(o)
        current_commentary.append(commentary_text[end:start+1][on])
    end=start+1
cricbuzz_commentary_df=pd.DataFrame({'innings':current_innings,'Over':current_over,'Commentary':current_commentary})
cricbuzz_commentary_df


,innings,Over,Commentary
0,New Zealand 2nd innings,70.6,"Labuschagne to Tom Blundell, out Caught by Lyo..."
1,New Zealand 2nd innings,70.5,"Labuschagne to Tom Blundell, no run, skids thr..."
2,New Zealand 2nd innings,70.4,"Labuschagne to Tom Blundell, FOUR, overpitches..."
3,New Zealand 2nd innings,70.3,"Labuschagne to Tom Blundell, no run, pitched u..."
4,New Zealand 2nd innings,70.2,"Labuschagne to Tom Blundell, no run, drops the..."
5,New Zealand 2nd innings,70.1,"Labuschagne to Tom Blundell, FOUR, drops it sh..."
6,New Zealand 2nd innings,69.6,"Lyon to Wagner, FOUR, that's smoked! Smash Fac..."
7,New Zealand 2nd innings,69.5,"Lyon to Tom Blundell, 1 run, tossed up on midd..."
8,New Zealand 2nd innings,69.4,"Lyon to Tom Blundell, no run, 95km/h, up from ..."
9,New Zealand 2nd innings,69.3,"Lyon to Tom Blundell, FOUR, another one that's..."


In [64]:
ib=[428,757,1090,2026]

inngs_breaks

[428, 757, 1090, 1091, 2026]

### Compile all tables

In [ ]:
##SNIPPET TO EXTRACT ALL WTC SCORECARD URLS OF CRICINFO

# page_url='https://www.espncricinfo.com/scores/series/19430/season/2019/icc-world-test-championship'
# r1=requests.get(page_url)
# bs_main=BeautifulSoup(r1.text,'html.parser')
# urllist=[]
# for link in bs_main.find_all('a',href=True,text='SCORECARD'):
#     urllist.append('https://www.espncricinfo.com'+link['href'])
# pd.DataFrame(urllist).to_csv('url.csv')


In [73]:
innings_state=pd.merge(cricbuzz_commentary_df,innings_fow_df,on=['innings','Over'],how='left').ffill()
last_fow_idx=min(innings_state[~pd.isnull(innings_state.active_partnership)].index)
##Process FOW for innings which were declared or not all out or won
if last_fow_idx>=1:
    last_fow_val=(innings_state[~pd.isnull(innings_state.active_partnership)]['active_partnership'])[last_fow_idx]+1
    innings_state.loc[0:last_fow_idx-1]['active_partnership']=last_fow_val
reviews_match=pd.merge(innings_state,idf,how='left')
reviews_match.fillna('',inplace=True)
reviews_match['match']=cricbuzz_match_url.split('/')[-1]
reviews_match
    
reviews_match.to_csv('data/reviews_match_{0}.csv'.format(cricbuzz_match_url.split('/')[-1]),index=False) 

### Read match wise Cricinfo and Cricbuzz URLs

In [10]:
url_list=pd.read_csv('review_data/URLs.csv')

url_list

,Cricinfo_URL,Cricbuzz_URL
0,https://www.espncricinfo.com/series/19430/scor...,https://www.cricbuzz.com/cricket-scores/20715/...
1,https://www.espncricinfo.com/series/19430/scor...,https://www.cricbuzz.com/cricket-scores/22954/...
2,https://www.espncricinfo.com/series/19430/scor...,https://www.cricbuzz.com/cricket-scores/20716/...
3,https://www.espncricinfo.com/series/19430/scor...,https://www.cricbuzz.com/cricket-scores/22955/...
4,https://www.espncricinfo.com/series/19430/scor...,https://www.cricbuzz.com/cricket-scores/20717/...
5,https://www.espncricinfo.com/series/19430/scor...,https://www.cricbuzz.com/cricket-scores/22859/...
6,https://www.espncricinfo.com/series/19430/scor...,https://www.cricbuzz.com/cricket-scores/22860/...
7,https://www.espncricinfo.com/series/19430/scor...,https://www.cricbuzz.com/cricket-scores/20718/...
8,https://www.espncricinfo.com/series/19430/scor...,https://www.cricbuzz.com/cricket-scores/20719/...
9,https://www.espncricinfo.com/series/19430/scor...,https://www.cricbuzz.com/cricket-scores/22743/...


### Debug

In [11]:
def compile_referral_data_debug(cricinfo_match_notes_url,cricbuzz_match_url):
    '''Compile all data from given cricinfo match notes url and cricbuzz match URL'''
    cricinfo_matchnotes_soup=scrape_cricinfo_match_notes(cricinfo_match_notes_url)
    cricbuzz_match_soup=parse_cricbuzz_commentary(cricbuzz_match_url)
    all_days=process_cricinfo_match_notes(cricinfo_matchnotes_soup)
    innings_df,innings_list=create_innings_df(all_days)
    return cricinfo_matchnotes_soup,cricbuzz_match_soup,all_days,innings_df,innings_list
    

### Run match by match [to avoid blocking of IPs]

In [13]:
url_list.iloc[15]

Cricinfo_URL    https://www.espncricinfo.com/series/19430/scor...
Cricbuzz_URL    https://www.cricbuzz.com/cricket-scores/22580/...
Name: 15, dtype: object

In [26]:
row=19
cricinfo_match_notes_url=(url_list.iloc[row]['Cricinfo_URL'])
cricbuzz_match_url=(url_list.iloc[row]['Cricbuzz_URL'])

reviews_match=compile_referral_data(cricinfo_match_notes_url,cricbuzz_match_url)
reviews_match.to_csv('review_data/reviews_match_{0}.csv'.format(cricbuzz_match_url.split('/')[-1]),index=False)
print("Match parsed",cricbuzz_match_url.split('/')[-1])

##FOR DEBUGGING USE BELOW SNIPPET

#cricinfo_matchnotes_soup,cricbuzz_match_soup,all_days,innings_df,innings_list=compile_referral_data_debug(cricinfo_match_notes_url,cricbuzz_match_url)

IndexError: list index out of range

### Collect all match wise referral DF into one

In [81]:
import glob

reviews_df=pd.DataFrame()

for glob in glob.glob("data/*.csv"):
    df=pd.read_csv(glob)
    reviews_df=pd.concat([df,reviews_df])
    
reviews_df.to_csv('data/reviews.csv')

C:\Users\k.shridhar\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [82]:
reviews_df=pd.read_csv('data/reviews.csv')
reviews_df['innings_action']=(reviews_df['Review_team'].astype('str')).apply(lambda x: "Batting" if "Batting" in x else "Bowling")

In [83]:
reviews_df.shape

(45217, 15)

### Process review events- to add overturned, review loss, 1st review loss, 2nd review loss logics

In [85]:
df=reviews_df.loc[(~pd.isnull(reviews_df.reviews))]

# df=df[df.match=='eng-vs-aus-3rd-test-the-ashes-2019']
# ##Split into innings
# reviews_df['Review_team']=reviews_df['Review_team'].astype('str')
# df=reviews_df
# df.fillna('',inplace=True)
##Keep start review count =2
## Overturn event logic. bowling-upheld and p.broken=True [OR] batting-upheld with p.broken=False 
##1st review lost at =
##2nd review lost at = 
## Review loss logic is take innings, take team batting- check upheld or struck down or struck down umpires call. add 1 if struck down
## for team bowling- check upheld or struck down or stuck down umpires call. if struck down, then add 1.

In [86]:
df['overturned']=df[['Review_team','Review_outcome','Partnership_broken']].apply(lambda x: True if 
(('Bowling' in x.Review_team and x.Review_outcome=='Upheld' and x.Partnership_broken==True) 
 or ('Batting' in x.Review_team and x.Review_outcome=='Upheld' and x.Partnership_broken==False)) else False,axis=1)

C:\Users\k.shridhar\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [87]:
df['review_lost']=df[['Review_team','Review_outcome','Partnership_broken']].apply(lambda x: True if 
(('Bowling' in x.Review_team and x.Review_outcome=='Struck down' and x.Partnership_broken==False) 
 or ('Batting' in x.Review_team and x.Review_outcome=='Struck down' and x.Partnership_broken==True)) else False,axis=1)


C:\Users\k.shridhar\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [88]:
df['first_review_lost']=(df.loc[df.review_lost==True].sort_values('Over').groupby(["match","innings","innings_action"])['review_lost'].rank('first')==1)

df['second_review_lost']=(df.loc[df.review_lost==True].sort_values('Over').groupby(["match","innings","innings_action"])['review_lost'].rank('first')==2)


C:\Users\k.shridhar\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\k.shridhar\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [89]:
df.fillna('',inplace=True)

c:\users\k.shridhar\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:4239: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


In [90]:
cols_to_add_from_df=['overturned','review_lost','first_review_lost','second_review_lost']

for col in cols_to_add_from_df:
    reviews_df[col]=df.loc[df.index,col]

In [91]:
reviews_df.fillna('',inplace=True)

In [ ]:
# reviews_df.to_csv('review_data/reviews_updated.csv')

In [92]:
reviews_df=pd.read_csv('review_data/reviews_updated.csv')

reviews_df.shape

(29481, 19)

### Visualisations for blog

- Batsman who took the most reviews (intiated by batsman)
- Whom did teams take a review against the most? 
- Umpire effectiveness (# of overturned decisions/total number of DRS calls for an umpire) Bar chart
- Review effectiveness by teams overall (Who had most partnership breaks when overturned)

In [ ]:
df=reviews_df[~pd.isnull(reviews_df.reviews)]

df.shape

print(df.columns)

### Overturned decisions per match

In [ ]:
overturned_per_match=df.groupby('match')['overturned'].sum()

### Total decisions per match

In [ ]:
total_per_match=df.groupby('match')['overturned'].count()

In [ ]:
ot_decisions=[]

tot_avg_decisions=[]

In [ ]:
for r in range((df.shape[0])):
    ot_decisions.append(overturned_per_match.loc[df.match.iloc[r]])
    tot_avg_decisions.append(overturned_per_match.loc[df.match.iloc[r]]/total_per_match.loc[df.match.iloc[r]])

In [ ]:
df['overturned_decisions']=ot_decisions

In [ ]:
df['ratio_overturned']=tot_avg_decisions

In [ ]:
# fig = px.bar(df,x='match',y='overturned_decisions')
# # fig.update_xaxes(categoryorder='sum descending')

# fig.show()

In [ ]:
df['team']=df['Review_team'].apply(lambda x:x.split('(')[0].strip())

df['bowler']=df['Commentary'].apply(lambda x:x.split('to')[0].strip())

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# df.to_csv('review_data/reviews_15dec.csv',index=False)

In [ ]:
df1=df.loc[df.review_lost==True]

In [ ]:
# df1.to_csv("review_data/rev_events.csv",index=False)

#### Fixes and enhancements:

- Active partnership sometimes increments additional fields- due to retired not outs and 1st innings declarations..etc.
- Add wicket keeper at review event